In [201]:
import re
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

sys.path.append(r'C:\Users\DerDo\Desktop\fantasy_basketball_project')
from functions import *

Players Career Data

In [46]:
soup = get_soup('https://www.basketball-reference.com/players/a/antetgi01.html')

In [93]:

### Individual Per Game data: Regular Season and Playoff

def player_avg_data(player_username):

    soup = get_soup(f'https://www.basketball-reference.com/players/{player_username[0]}/{player_username}.html')

    per_game_data = soup.select('table[data-soc-sum-table-type="PlayerPerGame"]')

    if not per_game_data:
        print('No Data Found')
        return

    ### Regular Season Table

    per_game_reg_season_columns = per_game_data[0].select('th[scope="col"]')
    per_game_reg_season_columns = [
        col.get_text(strip=True) for col in per_game_reg_season_columns
    ]

    per_game_reg_season_table_body = per_game_data[0].select('tbody')[0].select('th,td')
    table_data = []
    for i in per_game_reg_season_table_body:
        table_data.append(i.get_text(strip=True))

    reg_season_data = []
    BATCH_SIZE = 31
    for i in range(0, len(table_data), BATCH_SIZE):
        curr = table_data[i: i+BATCH_SIZE]
        reg_season_data.append(curr)

    reg_season = pd.DataFrame(reg_season_data)
    reg_season.columns = per_game_reg_season_columns

    reg_season = reg_season[
        reg_season.apply(lambda x: x != '')
    ].dropna(how='all', axis=0).fillna('')

    ### Playoff Table

    try:
        per_game_playoff_columns = per_game_data[1].select('th[scope="col"]')
    except IndexError as e:
        print('No Data Found')

    per_game_playoff_columns = [
        col.get_text(strip=True) for col in per_game_playoff_columns
    ]

    per_game_playoff_table_body = per_game_data[1].select('tbody')[0].select('th,td')
    table_data = []
    for i in per_game_playoff_table_body:
        table_data.append(i.get_text(strip=True))

    playoff_data = []
    BATCH_SIZE = 31
    for i in range(0, len(table_data), BATCH_SIZE):
        curr = table_data[i: i+BATCH_SIZE]
        playoff_data.append(curr)

    playoff_season = pd.DataFrame(playoff_data)
    playoff_season.columns = per_game_playoff_columns

    playoff_season = playoff_season[
        playoff_season.apply(lambda x: x != '')
    ].dropna(how='all', axis=0).fillna('')

    reg_season['Season Type'] = 'Regular'
    playoff_season['Season Type'] = 'Playoff'

    data = pd.concat([reg_season, playoff_season], axis=0)

    return data


In [88]:
def player_sum_data(player_username):
    
    soup = get_soup(f'https://www.basketball-reference.com/players/{player_username[0]}/{player_username}.html')

    total_data = soup.select('table[data-soc-sum-table-type="PlayerTotals"]')

    if not total_data:
        print('No Data Found')
        return

    ### Regular Season

    reg_data = total_data[0]
    reg_cols = [col.get_text() for col in reg_data.select('th[scope="col"]')]

    total_reg_season_table_body = reg_data.select('tbody')[0].select('th,td')
    table_data = []
    for i in total_reg_season_table_body:
        table_data.append(i.get_text(strip=True))

    reg_season_data = []
    BATCH_SIZE = len(reg_cols)
    for i in range(0, len(table_data), BATCH_SIZE):
        curr = table_data[i: i+BATCH_SIZE]
        reg_season_data.append(curr)

    reg_season = pd.DataFrame(reg_season_data)
    reg_season.columns = reg_cols

    reg_season = reg_season[
        reg_season.apply(lambda x: x != '')
    ].dropna(how='all', axis=0).fillna('')

    ### Playoffs

    try:
        post_data = total_data[1]
    except IndexError as e:
        print('No Data Found')
        return reg_season
        
    post_cols = [col.get_text() for col in post_data.select('th[scope="col"]')]

    total_post_season_table_body = post_data.select('tbody')[0].select('th,td')
    table_data = []
    for i in total_post_season_table_body:
        table_data.append(i.get_text(strip=True))

    post_season_data = []
    BATCH_SIZE = len(post_cols)
    for i in range(0, len(table_data), BATCH_SIZE):
        curr = table_data[i: i+BATCH_SIZE]
        post_season_data.append(curr)

    post_season = pd.DataFrame(post_season_data)
    post_season.columns = post_cols

    post_season = post_season[
        post_season.apply(lambda x: x != '')
    ].dropna(how='all', axis=0).fillna('')

    reg_season['Season Type'] = 'Regular'
    post_season['Season Type'] = 'Playoff'
    
    data = pd.concat([reg_season, post_season], axis=0)

    return data

In [87]:
def player_per36_data(player_username):

    soup = get_soup(f'https://www.basketball-reference.com/players/{player_username[0]}/{player_username}.html')

    table_data = soup.select('table[data-soc-sum-table-type="PlayerPerMinute"]')

    if not table_data:
        print('No Data Found')
        return

    ### Regular season

    reg_data = table_data[0]
    reg_cols = [
        col.get_text(strip=True) for col in reg_data.select('th[scope="col"]')
    ]

    reg_data = [
        row
        .get_text(strip=True) for row in reg_data.select('tbody')[0]
        .select('th, td')
    ]

    BATCH_SIZE = len(reg_cols)
    data = []
    for i in range(0, len(reg_data), BATCH_SIZE):
        curr_data = reg_data[i: i+BATCH_SIZE]
        data.append(curr_data)

    reg_season = pd.DataFrame(data)
    reg_season.columns = reg_cols

    reg_season = reg_season[
        reg_season.apply(lambda x: x != '')
    ].dropna(how='all', axis=0).fillna('')

    ### Playoffs

    try:
        post_data = table_data[1]
    except IndexError as e:
        print("No Playoff Data")
        return reg_season

    post_cols = [
        col.get_text(strip=True) for col in post_data.select('th[scope="col"]')
    ]

    post_data = [
        row
        .get_text(strip=True) for row in post_data.select('tbody')[0]
        .select('th, td')
    ]

    BATCH_SIZE = len(post_cols)
    data = []
    for i in range(0, len(post_data), BATCH_SIZE):
        curr = post_data[i: i+BATCH_SIZE]
        data.append(curr)

    post_season = pd.DataFrame(data)
    post_season.columns = post_cols

    post_season = post_season = post_season[
        post_season.apply(lambda x: x != '')
    ].dropna(how='all', axis=0).fillna('')

    reg_season['Season Type'] = 'Regular'
    post_season['Season Type'] = 'Playoff'

    data = pd.concat([reg_season, post_season], axis=0)

    return data

In [86]:
def player_advanced_data(player_username):

    soup = get_soup(f'https://www.basketball-reference.com/players/{player_username[0]}/{player_username}.html')

    table_data = soup.select('table[data-soc-sum-table-type="Advanced"]')

    if not table_data:
        print('No Data Found')
        return

    ### Regular season

    reg_data = table_data[0]
    reg_cols = [
        col.get_text(strip=True) for col in reg_data.select('th[scope="col"]')
    ]

    reg_data = [
        row
        .get_text(strip=True) for row in reg_data.select('tbody')[0]
        .select('th, td')
    ]

    BATCH_SIZE = len(reg_cols)
    data = []
    for i in range(0, len(reg_data), BATCH_SIZE):
        curr_data = reg_data[i: i+BATCH_SIZE]
        data.append(curr_data)

    reg_season = pd.DataFrame(data)
    reg_season.columns = reg_cols

    reg_season = reg_season[
        reg_season.apply(lambda x: x != '')
    ].dropna(how='all', axis=0).fillna('')

    ### Playoffs

    try:
        post_data = table_data[1]
    except IndexError as e:
        print("No Playoff Data")
        return reg_season

    post_cols = [
        col.get_text(strip=True) for col in post_data.select('th[scope="col"]')
    ]

    post_data = [
        row
        .get_text(strip=True) for row in post_data.select('tbody')[0]
        .select('th, td')
    ]

    BATCH_SIZE = len(post_cols)
    data = []
    for i in range(0, len(post_data), BATCH_SIZE):
        curr = post_data[i: i+BATCH_SIZE]
        data.append(curr)

    post_season = pd.DataFrame(data)
    post_season.columns = post_cols

    post_season = post_season = post_season[
        post_season.apply(lambda x: x != '')
    ].dropna(how='all', axis=0).fillna('')

    reg_season['Season Type'] = 'Regular'
    post_season['Season Type'] = 'Playoff'

    data = pd.concat([reg_season, post_season], axis=0)

    return data

In [103]:
def player_shooting_data(player_username):

    soup = get_soup(f'https://www.basketball-reference.com/players/{player_username[0]}/{player_username}.html')

    table_data = soup.select('table[data-soc-sum-table-type="Shooting"]')

    if not table_data:
        print('No Data Found')
        return

    ### Regular season

    reg_data = table_data[0]
    reg_cols = [
        col.get_text(strip=True) for col in reg_data.select('th[scope="col"]')
    ]

    reg_data = [
        row
        .get_text(strip=True) for row in reg_data.select('tbody')[0]
        .select('th, td')
    ]

    BATCH_SIZE = len(reg_cols)
    data = []
    for i in range(0, len(reg_data), BATCH_SIZE):
        curr_data = reg_data[i: i+BATCH_SIZE]
        data.append(curr_data)

    reg_season = pd.DataFrame(data)
    reg_season.columns = reg_cols

    reg_season = reg_season[
        reg_season.apply(lambda x: x != '')
    ].dropna(how='all', axis=0).fillna('')

    ### Playoffs

    try:
        post_data = table_data[1]
    except IndexError as e:
        print("No Playoff Data")
        return reg_season

    post_cols = [
        col.get_text(strip=True) for col in post_data.select('th[scope="col"]')
    ]

    post_data = [
        row
        .get_text(strip=True) for row in post_data.select('tbody')[0]
        .select('th, td')
    ]

    BATCH_SIZE = len(post_cols)
    data = []
    for i in range(0, len(post_data), BATCH_SIZE):
        curr = post_data[i: i+BATCH_SIZE]
        data.append(curr)

    post_season = pd.DataFrame(data)
    post_season.columns = post_cols

    post_season = post_season = post_season[
        post_season.apply(lambda x: x != '')
    ].dropna(how='all', axis=0).fillna('')

    reg_season['Season Type'] = 'Regular'
    post_season['Season Type'] = 'Playoff'

    shooting_data = pd.concat([reg_season, post_season], axis=0)

    return shooting_data

Player Season Data

In [104]:
soup = get_soup('https://www.basketball-reference.com/players/a/antetgi01/gamelog/2023/')

In [208]:
def player_season_data(player_username, season):    
    soup = get_soup(f'https://www.basketball-reference.com/players/{player_username[0]}/{player_username}/gamelog/{season}/')
    
    table = soup.select_one('table#player_game_log_reg')

    if not table:
        print("No Data Found")
        return

    header_cells = table.select('thead tr')[-1].select('th[scope="col"]')
    cols = [c.get_text(strip=True) for c in header_cells]
    NUM_COLS = len(cols)

    rows = []
    for tr in table.select('tbody tr'):
        if 'class' in tr.attrs and 'thead' in tr['class']:
            continue
        cells = tr.find_all(['th', 'td'])
        row = [c.get_text(strip=True) for c in cells]
        if len(row) < NUM_COLS:
            row += [''] * (NUM_COLS - len(row))
        elif len(row) > NUM_COLS:
            row = row[:NUM_COLS]
        rows.append(row)

    reg_season = pd.DataFrame(rows, columns=cols)
    reg_season['Season Type'] = 'Regular'

    ### Playoffs

    table = soup.select_one('table#player_game_log_post')

    if not table:
        print('No Playoff Data Found')

        reg_season = reg_season.drop(columns='')

        reg_season['TS'] = reg_season['Result'].apply(lambda x: re.findall(' [0-9]*', x)[0]).str.replace(' ', '')
        reg_season['OS'] = reg_season['Result'].apply(lambda x: re.findall('-[0-9]*', x)[0]).str.replace('-', '')
        reg_season['Result'] = reg_season['Result'].apply(lambda x: x[0])

        return reg_season
    
    rows = []
    for tr in table.select('tbody tr'):
        if 'class' in tr.attrs and 'thead' in tr['class']:
            continue
        cells = tr.find_all(['th', 'td'])
        row = [c.get_text(strip=True) for c in cells]
        if len(row) < NUM_COLS:
            row += [''] * (NUM_COLS - len(row))
        elif len(row) > NUM_COLS:
            row = row[:NUM_COLS]
        rows.append(row)

    post_season = pd.DataFrame(rows, columns=cols)
    post_season['Season Type'] = 'Playoff'

    data = pd.concat([reg_season, post_season], axis=0)
    data = data.drop(columns='')

    data['TS'] = data['Result'].apply(lambda x: re.findall(' [0-9]*', x)[0]).str.replace(' ', '')
    data['OS'] = data['Result'].apply(lambda x: re.findall('-[0-9]*', x)[0]).str.replace('-', '')
    data['Result'] = data['Result'].apply(lambda x: x[0])

    return data

In [211]:
player_data = player_season_data('murrake02', '2025')

No Playoff Data Found


In [212]:
player_data

,Rk,Gcar,Gtm,Date,Team,Opp,Result,GS,MP,FG,...,STL,BLK,TOV,PF,PTS,GmSc,+/-,Season Type,TS,OS
0,1,158,1,2024-10-24,SAC,MIN,L,*,43:00,8,...,0,0,0,3,23,18.6,9,Regular,115,117
1,2,159,2,2024-10-26,SAC,LAL,L,*,39:45,5,...,2,2,1,2,14,12.2,8,Regular,127,131
2,3,160,3,2024-10-28,SAC,POR,W,*,33:12,2,...,0,0,1,5,6,2.9,11,Regular,111,98
3,4,161,4,2024-10-29,SAC,UTA,W,*,32:21,6,...,1,0,1,1,14,8.8,16,Regular,113,96
4,5,162,5,2024-11-01,SAC,ATL,W,*,38:52,6,...,0,1,1,3,14,12.8,12,Regular,123,115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,74,,78,2025-04-06,SAC,CLE,W,Inactive,,,...,,,,,,,,Regular,120,113
78,74,,79,2025-04-07,SAC,DET,W,Inactive,,,...,,,,,,,,Regular,127,117
79,74,,80,2025-04-09,SAC,DEN,L,Inactive,,,...,,,,,,,,Regular,116,124
80,75,232,81,2025-04-11,SAC,LAC,L,*,31:54,3,...,2,0,1,1,9,8.8,7,Regular,100,101


In [188]:

player_data['Result']

0         W, 124-109
1         L, 122-133
2         L, 102-115
3         L, 108-119
4          L, 99-122
          ...       
0          L, 98-117
1         L, 115-123
2         W, 117-101
3         L, 103-129
4    L, 118-119 (OT)
Name: Result, Length: 87, dtype: object

In [ ]:
import re 



0    109
1    133
2    115
3    119
4    122
    ... 
0    117
1    123
2    101
3    129
4    119
Name: Result, Length: 87, dtype: object